In [194]:
from pprint import pprint
import os 

os.environ["GROQ_API_KEY"] = ""

In [195]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

In [196]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [197]:

def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [198]:
#Categorize EMAIL
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Email Categorizer Agent You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
        inquiry_handling - used when someone is asking for information about pricing \
        review_handling - used when someone is giving a review, either positive or negative \
        assistance_request_handling - used when someone is asking questions related to issues with equipment \\
        general_handling when it doesnt relate to any other category \


            Output a single cetgory only from the types ('inquiry_handling', 'review_handling', 'assistance_request_handling', 'general_handling') \
            eg:
            'inquiry_handling' \

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

email_category_generator = prompt | GROQ_LLM | StrOutputParser()

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

EMAIL = """HI there, \n
I am emailing to inquire about the Video camera. \n

Thanks,
Paul
"""

result = email_category_generator.invoke({"initial_email": EMAIL})

print(result)

'inquiry_handling'


In [199]:
# General Handling 
general_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an Expert Email Agent, write an email informing the customer that as this issue does not fit in the general categories we will be escalating this to the customer service for further evaluation.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    
    EMAIL CONTENT:\n\n {initial_email} \n\n
    EMAIL CATEGORY: \n\n {email_category} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
)

general_issue_chain = general_prompt | GROQ_LLM | StrOutputParser()

In [200]:
import sqlite3
from email.mime.text import MIMEText
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.document_loaders import TextLoader
from langchain_chroma.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

current_dir = os.path.dirname(os.path.abspath("/content"))
persistent_directory = os.path.join(current_dir, "data", "chroma_db_fe")

# Function to check availability and get price
def check_item_availability(item_name):
    conn = sqlite3.connect('data/film_equipment.db')  # Replace with your database connection details
    cursor = conn.cursor()

    # Query for the requested item
    cursor.execute("SELECT name, price FROM equipment WHERE name=?", (item_name,))
    result = cursor.fetchone()

    if result:
        item_price = result[1]
        return True, f"The item '{item_name}' is available at a price of ${item_price:.2f}."
    else:
        # Suggest similar items
        cursor.execute("SELECT equipment_id, name, type, brand, price FROM equipment")
        items = cursor.fetchall()
        item_texts = ""
        for item in items:
            item_texts += f"{item[1]} {item[2]} {item[3]} \n "
        f = open("data/items.txt", "w")
        f.write(item_texts)
        f.close()
        return False, items


# Function to search for similar items using LangChain and ChromaDB
def find_similar_items(item_name, items):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=20)
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    if not os.path.exists(persistent_directory):
        loader = TextLoader("data/items.txt")
        documents = loader.load()
        docs = text_splitter.split_documents(documents)

        vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=persistent_directory)

    else:
        vectorstore = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)

    # query_vector = embeddings.embed_query(item_name)

    results = vectorstore.similarity_search(item_name, k=5)
    suggestions = f"Sorry, the item '{item_name}' is not available. However, you might be interested in these similar items:\n"
    for i in range(len(results) - 1):
        suggestions += results[i].page_content + "\n"
    return suggestions

def handle_inquiry(item_name):
    
    is_available, result = check_item_availability(item_name)

    if is_available:
        response = result
    else:
        response = find_similar_items(item_name, result)

    return response




In [201]:
## Search keywords
search_keyword_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a master at identifying the film equipments to search for in a database to get the best info for the customer.

    given the INITIAL_EMAIL and EMAIL_CATEGORY. Extract the film equipments which are in the email.

    Return just the film equipment found in the email.

    eg:
        'Video camera',
        'Camera

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)

search_keyword_chain = search_keyword_prompt | GROQ_LLM | StrOutputParser()

email_category = 'inquiry_handling'
research_info = None

print(search_keyword_chain.invoke({"initial_email": EMAIL, "email_category":email_category}))

'Video camera'


In [202]:


inquiry_handler = search_keyword_chain | handle_inquiry 

email_category = 'customer_feedback'

print(inquiry_handler.invoke({"initial_email": EMAIL, "email_category":email_category}))

c:\Users\raahu\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Sorry, the item ''Video camera'' is not available. However, you might be interested in these similar items:
Updated Camera Video Canon 
 Camera Digital Canon
Cage Camera SmallRig
Cage Camera SmallRig
Wireless Video Transmission Teradek



In [203]:
review_handler_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at reading the initial email and classify whether the review is positive or negative. If the reivew is positive then Thank the sender and encourage them to share their experience on social media. And if the review is negative then  Escalate to the CRM system for follow-up with a phone call from customer service and offer a gift voucher in the reply.\n

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Email to route INITIAL_EMAIL : {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)
review_handler_chain = review_handler_prompt | GROQ_LLM | StrOutputParser()

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""
email_category = 'review_handling'
research_info = None

print(review_handler_chain.invoke({"initial_email": EMAIL, "email_category":email_category}))

Dear Paul,

Thank you so much for taking the time to share your wonderful experience at our resort! We're thrilled to hear that our staff made a positive impact on your stay. 

We're grateful for customers like you and would love to hear more about your experience. Please consider sharing your review on social media to help others discover the great service we provide.

Once again, thank you for your kind words, and we look forward to welcoming you back soon!

Best regards,
[Your Name]


In [204]:
import os
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def get_vector_store():
    # Define the directory containing the text file and the persistent directory
    current_dir = os.path.dirname(os.path.abspath("C:/Users/raahu/OneDrive/Documents/_Workspace/YOLO/AutomaticEmailReplySystem/src/AutomaticEmailGeneration.ipynb"))
    file_path = os.path.join(current_dir, "data", "fe_faq.txt")
    persistent_directory = os.path.join(current_dir, "data", "chroma_db_faq")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    print("\n--- Finished Creating embeddings ---")

    if not os.path.exists(persistent_directory):
        print("Persistent directory does not exist, Initializing vector store")
        if not os.path.exists(file_path):
            raise FileNotFoundError(
                f"The file {file_path} does not exist. Please check the path. "
            )
        loader = TextLoader(file_path)
        documents = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
        docs = text_splitter.split_documents(documents)
        print("\n--- Document splitting done ---")

        vectorstore = Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory
        )
        print("\n--- Finished Creating a Vector Store ---")

    else:
        vectorstore = Chroma(embedding_function=embeddings, persist_directory=persistent_directory)

    return vectorstore


vectorstore = get_vector_store()
retriever = vectorstore.as_retriever(
search_type="similarity_score_threshold",
search_kwargs={"k": 3, "score_threshold": 0.05}
)

def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)


rag_prompt = hub.pull("rlm/rag-prompt")

# Create the RAG chain using the pipe operator
rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | rag_prompt
        | GROQ_LLM
        | StrOutputParser()
)

# Execute the chain
response = rag_chain.invoke("I have an issue with the camera")



c:\Users\raahu\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



--- Finished Creating embeddings ---


c:\Users\raahu\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\vectorstores\base.py:784: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': 'C:\\Users\\raahu\\OneDrive\\Documents\\_Workspace\\YOLO\\AutomaticEmailReplySystem\\src\\data\\fe_faq.txt'}, page_content='What is a wide-angle lens?\n\n    A wide-angle lens has a short focal length, typically 35mm or less, allowing you to capture a broader field of view.\n\nWhat is a telephoto lens?\n\n    A telephoto lens has a long focal length, typically 70mm or more, allowing you to capture distant subjects with magnification.\n\nWhat is the best lens for shooting landscapes?\n\n    Wide-angle lenses, such as a 16-35mm, are ideal for capturing expansive landscapes.\n\nWhat is the best lens for shooting portraits?\n\n    A 50mm or 85mm prime lens with a wide aperture (f/1.4 or f/1.8) is great for portrait photography.\n\nWhat is the best lens for shooting action scenes?\n\n    A 

In [205]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [206]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_email: email
        email_category: email category
        draft_email: LLM generation
        final_email: LLM generation
        research_info: list of documents
        info_needed: whether to add search info
        num_steps: number of steps
    """
    initial_email : str
    email_category : str
    response_: str
    num_steps : int

In [207]:
def categorize_email(state):
    """take the initial email and categorize it"""
    print("---CATEGORIZING INITIAL EMAIL---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email_category = email_category_generator.invoke({"initial_email": initial_email})
    print(email_category)
    # save to local disk
    write_markdown_file(email_category, "email_category")

    return {"email_category": email_category, "num_steps":num_steps}

In [208]:
def email_handling(state):
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    num_steps = state['num_steps']
    num_steps += 1

    if email_category == "inquiry_handling":
        response_ = inquiry_handler.invoke({"initial_email":initial_email, "email_category":email_category})
        write_markdown_file(response_ , "response_")
    
    elif email_category == "review_handling":
        response_ = review_handler_chain.invoke({"initial_email": initial_email, "email_category": email_category})
        write_markdown_file(response_, "response_")

    elif email_category == "assistance_request_handling":
        response_ = rag_chain.invoke(initial_email)
        write_markdown_file(response_, "response_")
    
    else:
        response_ = general_issue_chain.invoke({"initial_email": initial_email, "email_category": email_category})
        write_markdown_file(response_, "response_")


    return {"response_": response_, "num_steps":num_steps}


In [209]:
def state_printer(state):
    """print the state"""
    print("---STATE PRINTER---")
    print(f"Initial Email: {state['initial_email']} \n" )
    print(f"Email Category: {state['email_category']} \n")
    print(f"Response : {state['response_']} \n")
    print(f"Num Steps: {state['num_steps']} \n")
    return

In [210]:
workflow = StateGraph(GraphState)
# Define the nodes
workflow.add_node("categorize_email", categorize_email)
workflow.add_node("email_handling", email_handling)  
workflow.add_node("state_printer", state_printer)

In [211]:
workflow.add_edge("categorize_email", "email_handling")
workflow.add_edge("email_handling", "state_printer")


### Add Edges

In [212]:
workflow.set_entry_point("categorize_email")


workflow.add_edge("state_printer", END)

In [213]:
# Compile
app = workflow.compile()

In [214]:
# EMAIL = """HI there, \n
# I am emailing to find out the current price of Bitcoin. \n

# Can you please help me/

# Thanks,
# John
# """

# EMAIL = """HI there, \n
# I am emailing to say that I had a wonderful stay at your resort last week. \n

# I really appreaciate what your staff did

# Thanks,
# Paul
# """

# EMAIL = """HI there, \n
# I am emailing to say that the resort weather was way to cloudy and overcast. \n
# I wanted to write a song called 'Here comes the sun but it never came'

# What should be the weather in Arizona in April?

# I really hope you fix this next time.

# Thanks,
# George
# """

EMAIL = """HI there, \n
I have an issue with the Camera

Thanks,
Ringo
"""

# EMAIL = """HI there, \n
# Hey I wanna talk about some general issue i Have been facing

# Thanks,
# Ringo
# """

In [215]:
output = app.invoke(inputs)

print(output['response_'])

---CATEGORIZING INITIAL EMAIL---
'general_handling'
---STATE PRINTER---
Initial Email: HI there, 

Hey I wanna talk about some general issue i Have been facing

Thanks,
Ringo
 

Email Category: 'general_handling' 

Response : Here is an email response:

Dear Ringo,

Thank you for reaching out to us about the general issue you've been experiencing. I appreciate you taking the time to share this with us.

After reviewing your concern, I realize that it doesn't fit into our general categories. To ensure you receive the best possible assistance, I'm going to escalate this issue to our Customer Service team for further evaluation.

They will review your case and provide a more personalized solution. You can expect a response from them within the next 24-48 hours.

Thank you for your patience and cooperation. If you have any further questions or concerns, please don't hesitate to reach out.

Best regards,
[Your Name]
Email Support Agent 

Num Steps: 2 

Here is an email response:

Dear Ringo